<h2>This script is for internal developer testing.</h2>

In [ ]:
import numpy as np
from wfdb import readsignal
# Testing rdsamp format 212
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=100, sampto=9999) # Default physical values
print(sig)
print(len(sig))

sig, fields=readsignal.rdsamp('sampledata/100', physical=0) # Digital values
print(sig)

In [ ]:
# Testing rdsamp format 16
sig, fields=readsignal.rdsamp('sampledata/test01_00s')
print(sig)
print(fields)


In [ ]:
# Testing rdsamp format 80
sig, fields=readsignal.rdsamp('sampledata/3000003_0003')
print(sig)
print(fields)

In [ ]:
# Testing rdsamp format 16 with byte offset (actually a .mat file too)
import numpy as np
from wfdb import readsignal
sig, fields=readsignal.rdsamp('sampledata/a103l') 
#print(sig)
#print(sig.shape)
#print(fields)

from wfdb import plotwfdb

plotwfdb.plotsigs(sig, fields)


# With channel selection and sampfrom sampto
#sig, fields=readsignal.rdsamp('sampledata/a103l', channels=[2,0], sampfrom=250, sampto=80000) 
#print('\n\n')
#print(sig)
#print(fields)

In [ ]:
# Testing rdsamp multi-segment variable layout record (from https://physionet.org/physiobank/database/mimic2wdb/matched/s00001/)
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=3200, sampto=20000, stacksegments=0)
#print("len(sig): ", len(sig))
#print(len(fields))

print(sig)
print("\n\n")
print(sig[len(sig)-1])
print("\n\n")
print(len(sig[len(sig)-1]))
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-segment variable layout record - stacksegments
import numpy as np
from wfdb import readsignal


# Return single numpy array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', stacksegments=1, sampto=10000, channels=[1, 2])

print(sig)
print(sig.shape)
print("\n\n")
print(fields[0])
print("\n\n")
print(fields[1])
print("\n\n")
print(fields[2])

In [ ]:
# Testing rdsamp multi-dat signals (from https://physionet.org/physiobank/database/ptbdb/patient001/)
import numpy as np
from wfdb import readsignal

sig, fields=readsignal.rdsamp('sampledata/s0010_re', channels=[14, 0, 5, 10, 11], sampfrom=100, sampto=20000)
from wfdb import plotwfdb
plotwfdb.plotsigs(sig, fields)
print("\n\n\n", sig, "\n\n\n")
print(fields)
print(fields["signame"])

In [ ]:
# Testing rdsamp multi-segment record with sampfrom, sampto, and channel selection
import numpy as np
from wfdb import readsignal

# Return list of arrays
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[2,3], stacksegments=0)

print(sig)
# Return single array
sig, fields=readsignal.rdsamp('sampledata/multisegmentV/s00001-2896-10-10-00-31', sampfrom=10, sampto=10200, channels=[1,0,2],stacksegments=1)

print(sig)

<h2>Decide comprehensive tests before adding to travis</h2>
Target files created with wfdb software package's <code>rdsamp</code>

In [ ]:
import numpy as np
from wfdb import readsignal

# Test 1 - Format 212/Entire signal/Physical 
# Target file created with: rdsamp -r sampledata/100 -P | cut -f 2- > target1
sig, fields=readsignal.rdsamp('sampledata/100') 

# Test 2 - Format 212/Selected Duration/Selected Channel/Digital
# Target file created with: rdsamp -r sampledata/100 -f 1 -t 30 -s 1 | cut -f 2- > target2
sig, fields=readsignal.rdsamp('sampledata/100', sampfrom=360, sampto=10800, channels=[1], physical=0)  

# Test 3 - Format 16/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/test01_00s | cut -f 2- > target3
sig, fields=readsignal.rdsamp('sampledata/test01_00s', physical=0)

# Test 4 - Format 16 with byte offset/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/a103l -f 50 -t 160 -s 2 0 -P | cut -f 2- > target4
sig, fields=readsignal.rdsamp('sampledata/a103l', sampfrom=12500, sampto=40000, channels=[2, 0])

# Test 5 - Format 16 with byte offset/Selected Duration/Selected Channels/Digital
# Target file created with: rdsamp -r sampledata/a103l -f 80 -s 0 1 | cut -f 2- > target5
sig, fields=readsignal.rdsamp('sampledata/a103l', sampfrom=20000, physical=0, channels=[0, 1])

# Test 6 - Format 80/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/3000003_0003 -f 1 -t 8 -s 1 -P | cut -f 2- > target6
sig, fields=readsignal.rdsamp('sampledata/3000003_0003', sampfrom=125, sampto=1000, channels=[1])

# Test 7 - Multi-dat/Entire signal/Digital
# Target file created with: rdsamp -r sampledata/s0010_re | cut -f 2- > target7
sig, fields=readsignal.rdsamp('sampledata/s0010_re', physical=0)

# Test 8 - Multi-dat/Selected Duration/Selected Channels/Physical
# Target file created with: rdsamp -r sampledata/s0010_re -f 5 -t 38 -P -s 13 0 4 8 3 | cut -f 2- > target8
sig, fields=readsignal.rdsamp('sampledata/s0010_re', sampfrom=5000, sampto=38000, channels=[13, 0, 4, 8, 3])

In [ ]:
import numpy as np
from wfdb import readannot

readannot.rdann('sampledata/100', 'atr')

In [ ]:
x=[[  0  88] 0
 [ 23 252]1 
 [ 35  35]2
 [ 32 116]3
 [105 109]4
 [101  32]5
 [114 101]6
 [115 111]7
 [108 117]8
 [116 105]9
 [111 110]10
 [ 58  32]11
 [ 51  54]12
 [ 48   0]13
 [  0 236]14
 [255 255]15
 [255 255]16
 [  1   0]17
 [ 18 112]18
 [ 59   4]19
 [  0 236]20
 [  0   0]21
 [ 16  39]22
 [  0   4]23
 [  0 236]24
 [  0   0]25
 [ 16  39]26
 [  0   4]27
 [  4 252]28
 [ 75  75]29
 [ 75  75]30
 [100   4]31
 [  4 252]32
 [ 65  65]33
 [ 65   9]34
 [100   4]35
 [  3 252]36
 [ 66  67]37
 [ 69   0]38
 [  0   0]]39



In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/100', 'fake2')
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)

In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/chb01_03.edf', 'seizures')
print(annsamp)
print(anntype)
print(num)
print(subtype)
print(chan)
print(aux)
print(annfs)

In [ ]:
from wfdb import readannot
annsamp, anntype, num, subtype, chan, aux, annfs=readannot.rdann('sampledata/100', 'fake3')
print("annsamp: ", annsamp)
print("anntype: ", anntype)
print("num: ", num)
print("subtype: ", subtype)
print("chan: ", chan)
print("aux: ", aux)
print("annfs: ", annfs)